In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

# Process Data

In [2]:
#!/home/lgunthermann/miniconda3/envs/discushack22/bin/python3 process_csv.py

In [3]:
data_coding = pd.read_csv('data/processed/coding_both.csv')
data_invoice = pd.read_csv('data/processed/invoices.csv')

In [4]:
df = pd.DataFrame().reindex(columns=data_coding.columns)
df = df.rename(columns={"Expense Category": "Expense Category (Coding)"})
df.insert(1,'ECRef',[])
df.insert(6,'Expense Category (Invoice)',[])
df.drop(['ID'], axis=1)

,ECRef,Supplier,Party,Total Spend,Expense Category (Coding),Expense Category (Invoice),Transport,Miscellaneous,Catering,Accommodation,...,Legal advice,Research,Polling,Focus groups,Ordinance survey data,Message testing,Archival research,Other forms of research,Social/Digital listening,Phone banking


# Output Dataframe

The new dataframe consists of 64 columns. 
- ECRef linking the entry to the invoice list
- Supplier
- Party
- Total Spend
- Expense Category (According to the coding list)
- Expense Category (According to the invoice list)
- 58 expense categorisations

In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def print_positions(party,supplier):
    print("Invoices:")
    invoices = data_invoice.loc[ (data_invoice['RegulatedEntityName'] == party) & (data_invoice['SupplierName'] == supplier) ]
    display(invoices)
    print("Sum TotalExpenditure:",invoices['TotalExpenditure'].sum())
    print("\nCoding:")
    display(data_coding.loc[ (data_coding['Party'] == party) & (data_coding['Supplier'] == supplier) ])
    
def print_supplier(supplier):
    print("Invoices:")
    invoices = data_invoice.loc[ (data_invoice['SupplierName'] == supplier) ]
    display(invoices)
    print("Sum TotalExpenditure:",invoices['TotalExpenditure'].sum())
    print("\nCoding:")
    display(data_coding.loc[ (data_coding['Supplier'] == supplier) ])
    
def get_nth_key(dictionary, n=0):
    if n < 0:
        n += len(dictionary)
    for i, key in enumerate(dictionary.keys()):
        if i == n:
            return key
    raise IndexError("dictionary index out of range") 

In [6]:
#display(data_coding)

# Data Preview

In [7]:
data_coding.head()

,ID,Supplier,Party,Total Spend,Expense Category,Transport,Miscellaneous,Catering,Accommodation,Expenses claimed by provider,Completely unclear,Ambiguous,Advertising and Press,Merchandise,Newspaper or magazine advertising,Radio advertising,Social media advertising,Online advertising,Other forms of advertising,PR,Campaign materials,Design services,Campaign material printing,Postage,Infrastructure and equipment,Telecommunications services,Physical Security,Event costs/Venue hire,Mobile application services,Email services,Website services,Search Engine Optimization,Recruitment services/staffing costs,Creative content,Translation/Braile/British Sign Language services,Campaign activity,GOTV,Fundraising,Data and infrastructure,Campaign database or CRM,Data Services and analysis,IT infrastructure and support,Office supplies,Production Services,Video editing/production,Audio editing/production,Photos editing/production,Consultancy,Communication consultants,Design consultants,Ad strategy and consultancy,Social media strategy and consultancy,Data consultancy,Legal advice,Research,Polling,Focus groups,Ordinance survey data,Message testing,Archival research,Other forms of research,Social/Digital listening,Phone banking
0,0,Abi Bell,Liberal Democrats,1936.07,Transport,0.0,0.0,0.0,0.0,0.0,1936.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Abigail Baker,Liberal Democrats,748.80,Transport,0.0,0.0,0.0,0.0,0.0,748.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Adam Prices,Plaid Cymru,275.85,Transport,0.0,0.0,0.0,0.0,0.0,275.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Adam Williams,Liberal Democrats,484.06,Transport,0.0,0.0,0.0,0.0,0.0,484.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Addison Lee,Conservative Party,22648.35,Transport,0.0,0.0,0.0,0.0,0.0,22648.35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
data_invoice.head()

,ECRef,RegulatedEntityName,ReportingPeriodName,RegulatedEntityType,TotalExpenditure,DateIncurred,ExpenseCategoryName,SupplierName,FullAddress,AmountInEngland,AmountInScotland,AmountInWales,AmountInNorthernIreland,DateOfClaimForPayment,DatePaid,RedactedSupportingInvoiceId,RegulatedEntityId,SupplierId,JointCampaignName,UnregisteredCampaignerName,CampaigningName,IsOutsideSection75
0,SP0555253,Plaid Cymru,UK Parliamentary general election 12/12/2019,Political Party,200.40,03/01/2019,Unsolicited material to electors,Low Cost Printing,"Precision House, Pentrpoeth Rd, Swansea, SA6 6AA, United Kingdom",£0.00,£0.00,£200.40,£0.00,NaN,03/01/2019,74031,77,2276,NaN,NaN,NaN,False
1,SP0541026,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,60.00,01/04/2019,Advertising,Rayleigh Town Council,"The Pavilion, KGV Playing Field, Bull Lane, Rayleigh, SS6 8JD, United Kingdom",£60.00,£0.00,£0.00,£0.00,01/04/2019,10/05/2019,0,90,68727,NaN,NaN,NaN,False
2,SP0541025,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,36.00,17/06/2019,Market research/canvassing,Rochford District Council,"Council Offices, South Street, ROCHFORD, SS4 1BW, United Kingdom",£36.00,£0.00,£0.00,£0.00,17/06/2019,04/07/2019,0,90,76767,NaN,NaN,NaN,False
3,SP0541024,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,9.99,07/01/2019,Advertising,RelyOn IT (UK),"1386 London Rd, Leigh-on-Sea, SS9 2UJ, United Kingdom",£9.99,£0.00,£0.00,£0.00,07/01/2019,10/01/2019,0,90,76765,NaN,NaN,NaN,False
4,SP0541023,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,72.00,18/01/2019,Unsolicited material to electors,Off-Tech,"1 Brodie Road, ENFIELD, ZZ0 0ZZ, United Kingdom",£72.00,£0.00,£0.00,£0.00,18/01/2019,18/02/2019,0,90,76766,NaN,NaN,NaN,False


# Test

In [9]:
def get_totals(invoice_amounts,total_values,prediction):
    totals = np.zeros_like(total_values)
    for amount, cat in zip(invoice_amounts,prediction):
        totals[cat] += amount
    return totals

def get_error(invoice_amounts,total_values,prediction,loss='MSE'):
    ''' MSE or MAE loss '''
    totals = get_totals(invoice_amounts,total_values,prediction)
    
    if loss == 'MSE':
        return np.mean(np.square(total_values-totals))
    else:
        return np.mean(np.abs(total_values-totals))
    
def random_solver(invoice_amounts,total_values,loss='MSE'):
    prediction = np.random.randint(low=0, high=len(total_values), size=invoice_amounts.shape[0], dtype=int)
    
    return prediction

In [10]:
parties = pd.concat((data_coding['Party'], data_invoice['RegulatedEntityName'])).unique()
categories = data_coding.columns[5:]

error = 0
row_list = []

for party in parties:
    #print(f"Party: {party}")
    
    party_coding = data_coding[ data_coding['Party']==party ]
    party_invoice = data_invoice[ data_invoice['RegulatedEntityName']==party ]
    
    suppliers = pd.concat(( party_coding['Supplier'], party_invoice['SupplierName'] )).unique()
    
    for supplier in suppliers:
        sup_coding = party_coding[ party_coding['Supplier']==supplier ]
        sup_invoice = party_invoice[ party_invoice['SupplierName']==supplier ]
        
        sup_cats = {}
        
        for cat in categories:
            val = np.sum(sup_coding[cat].values)
            if val >= 0.01:
                sup_cats[cat] = val
        
        if len(sup_cats) < 1:
            continue
        
        spent = np.round(sup_coding['Total Spend'].sum(),2)
        filed = np.round(sup_invoice['TotalExpenditure'].sum(),2)
        
        # random encoding which invoice belongs to which category
        values = np.fromiter(sup_cats.values(), dtype=float)
        invoices = sup_invoice['TotalExpenditure'].values
        prediction = random_solver(invoices,values,loss='MSE')
        
        error += get_error(invoices,values,prediction,loss='MAE')
        
        if False:
            print(f"Diff = {abs(spent-filed)} ({spent=} | {filed=}) accross {len(sup_cats)} categories with {sup_coding.shape[0]} codings and {sup_invoice.shape[0]} invoices.")

            print("\nCoding:")
            display(sup_coding)

            print("\nInvoice:")
            display(sup_invoice)

        for i, (idx, row) in enumerate(sup_invoice.iterrows()):
            val = row['TotalExpenditure']
            entry = {
                'ECRef': row['ECRef'],
                'Supplier': supplier,
                'Party': party,
                'Total Spend': val,
                'Expense Category (Coding)': sup_coding['Expense Category'].mode(),
                'Expense Category (Invoice)': row['ExpenseCategoryName'],
                **{k:0 for k in categories}
                    }

            cat = get_nth_key(sup_cats, n=prediction[i])
            entry[cat]=val
            
            row_list.append(entry)
            
df = pd.DataFrame.from_dict(row_list)
df.head()

Party: Liberal Democrats
Party: Plaid Cymru
Party: Conservative Party
Party: Labour Party
Party: Scottish National Party
Party: Green Party
Party: Reform UK
Party: The Independent Group for Change
Party: Christian Peoples Alliance
Party: Scottish Family Party
Party: Animal Welfare Party
Party: The Universal Good Party
Party: UKIP
Party: Womens Equality Party
Party: Co-op Party
Party: Social Democratic Party
Party: Sinn Féin
Party: The Cynon Valley Party [De-registered 06/11/20]
Party: The Socialist Party of Great Britain
Party: SDLP (Social Democratic & Labour Party)
Party: Advance Together [De-registered 01/10/20]
Party: Democratic Unionist Party - D.U.P.
Party: Alliance - Alliance Party of Northern Ireland
Party: Ashfield Independents
Party: Christian Party
Party: Ulster Unionist Party
Party: English Democrats
Party: Proud of Oldham & Saddleworth
Party: Lincolnshire Independents Lincolnshire First
Party: Church of the Militant Elvis
Party: Libertarian Party
Party: Burnley and Padiham

,ECRef,Supplier,Party,Total Spend,Expense Category (Coding),Expense Category (Invoice),Transport,Miscellaneous,Catering,Accommodation,Expenses claimed by provider,Completely unclear,Ambiguous,Advertising and Press,Merchandise,Newspaper or magazine advertising,Radio advertising,Social media advertising,Online advertising,Other forms of advertising,PR,Campaign materials,Design services,Campaign material printing,Postage,Infrastructure and equipment,Telecommunications services,Physical Security,Event costs/Venue hire,Mobile application services,Email services,Website services,Search Engine Optimization,Recruitment services/staffing costs,Creative content,Translation/Braile/British Sign Language services,Campaign activity,GOTV,Fundraising,Data and infrastructure,Campaign database or CRM,Data Services and analysis,IT infrastructure and support,Office supplies,Production Services,Video editing/production,Audio editing/production,Photos editing/production,Consultancy,Communication consultants,Design consultants,Ad strategy and consultancy,Social media strategy and consultancy,Data consultancy,Legal advice,Research,Polling,Focus groups,Ordinance survey data,Message testing,Archival research,Other forms of research,Social/Digital listening,Phone banking
0,SP0528871,Abi Bell,Liberal Democrats,26.60,"0 Transport Name: Expense Category, dtype: object",Transport,0.0,0.0,0.0,0.0,0.0,26.60,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,SP0528867,Abi Bell,Liberal Democrats,30.69,"0 Transport Name: Expense Category, dtype: object",Transport,0.0,0.0,0.0,0.0,0.0,30.69,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,SP0528866,Abi Bell,Liberal Democrats,61.38,"0 Transport Name: Expense Category, dtype: object",Transport,0.0,0.0,0.0,0.0,0.0,61.38,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
3,SP0528865,Abi Bell,Liberal Democrats,57.87,"0 Transport Name: Expense Category, dtype: object",Transport,0.0,0.0,0.0,0.0,0.0,57.87,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,SP0528864,Abi Bell,Liberal Democrats,63.81,"0 Transport Name: Expense Category, dtype: object",Transport,0.0,0.0,0.0,0.0,0.0,63.81,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [11]:
print("Error:",error)

Error: 6182830.409031735
